<a href="https://colab.research.google.com/github/VedangJotaniya/CP441_Project_Repo/blob/gh-pages/RNN_Model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Model Creation**

In [55]:
import numpy
import cv2
from glob import glob
import argparse
from google.colab.patches import cv2_imshow

from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.layers import Flatten, Dense, Dropout, Input

from tensorflow.keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, TimeDistributed, LSTM
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense



In [56]:
IMG_HEIGHT=IMG_WIDTH=224

ResNet = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

for layer in ResNet.layers:
  layer.trainable = False


In [57]:
Timesteps = 5

model = Sequential()

model.add(Input(shape = (5, IMG_HEIGHT,IMG_WIDTH,3)))
model.add(TimeDistributed(ResNet))
model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(32, activation='relu')))
model.add(LSTM(16, return_sequences=False))
model.add(Dense(3, activation='sigmoid'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [58]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_3 (TimeDist (None, 5, 7, 7, 2048)     23587712  
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 100352)         0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 5, 32)             3211296   
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 51        
Total params: 26,802,195
Trainable params: 3,214,483
Non-trainable params: 23,587,712
_________________________________________________________________


##**Import and prepare DATASET**

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [61]:
def load_data(DataPath, CATEGORIES, IMAGE_SIZE, sampling_rate):
  Data = []
  for category in CATEGORIES:
    path = os.path.join(DataPath, category)
    class_num = CATEGORIES.index(category)
    for vid in os.listdir(path):
      ctr = 0
      temp = []
      try:
        video = cv2.VideoCapture(os.path.join(path, vid))
        length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        
        while(video.isOpened()):
          ret, frame = video.read()
          if ret:
            if ctr%sampling_rate == 0:
              frame = cv2.resize(frame, (IMAGE_SIZE, IMAGE_SIZE))
              temp.append(frame)
            ctr+=1
          else:
            break
        video.release()
        Data.append([np.array(temp), class_num])
        
      except Exception as e:
        print("Exception is raised")
  
  return Data


In [62]:
folder_path = "drive/My Drive/Colab Notebooks/Data/Words/"
CATEGORIES = ['Father','Mother','Sunday']

image_size = 224

data = load_data(folder_path, CATEGORIES, image_size, 2)

X_train = []
X_test = []
y_train = []
y_test = []

In [63]:
print(len(data[0][0]))

36


In [ ]:
print(data[0][1])
for f in data[0][0]:
  cv2_imshow(f)


In [64]:
import random

random.shuffle(data)

print(len(data[0][0]))

26


Generating Train set and Test set

In [65]:
from sklearn.preprocessing import OneHotEncoder

for features, label in data:
  X_train.append(np.array(features))
  y_train.append(label)

X_train = np.array(X_train)
X_train = X_train/255

y_train = np.array(y_train)
OneHot = OneHotEncoder()
y_train = OneHot.fit_transform(y_train.reshape(-1,1)).toarray()





Training the Model

In [66]:
model.fit(X_train, y_train, epochs=1)

ValueError: ignored